In [4]:
import pandas as pd

In [5]:
f = pd.read_csv('../Raw_Datasets/items_Giacomo/items.csv',sep=',')
f['TIPO'].unique()

array(['FUENTE', 'INSTALACIÓN DEPORTIVA', 'PISCINA', 'ZONA VERDE',
       'SENDA', 'CENTRO DE SALUD', 'CICLOCARRIL', 'PASO DE BICI',
       'CARRIL BICI', 'VÍA MIXTA', 'SENDA CICLABLE', 'PISTA BICI',
       'ARCEN BICI', 'ACERA BICI', 'ENCAMINAMIENTO', 'CIRCUITO DE OCIO',
       'BANDA RECOMENDADA', 'GIRO PERMITIDO', 'SENTIDO PERMITIDO'],
      dtype=object)

In [6]:
import json
import csv

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8',newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

def obtener_recomendaciones(datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud):
    # Implementa la lógica para generar las recomendaciones de deportes
    # basadas en las condiciones climáticas, la ubicación del usuario y los datos
    # de instalaciones deportivas disponibles.

    recomendaciones = []

    # Implementa la lógica para recomendar deportes seguros según si hay ola de calor o no.
    if etiquetas_ola_calor[0] == 1:  # Primer registro en los datos de etiquetas corresponde a la predicción actual
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'].lower() == 'centro_salud' and tiene_problemas_salud == True:
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    recomendaciones_deportes = obtener_recomendaciones(
        datos_clima, etiquetas_ola_calor, datos_instalaciones, tiene_problemas_salud
    )

    # Imprime las recomendaciones
    print("Recomendaciones de deportes:")
    for deporte in recomendaciones_deportes:
        print(deporte)


Recomendaciones de deportes:


In [7]:
import json
import csv
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

# Función para leer datos meteorológicos desde un archivo JSON
def leer_datos_meteorologicos(nombre_archivo):
    with open(nombre_archivo, 'r') as archivo:
        datos = json.load(archivo)
    return datos

# Función para etiquetar la presencia de una ola de calor basada en datos climáticos
def etiquetar_ola_de_calor(datos_clima, umbral_temperatura):
    etiquetas = []
    for registro in datos_clima['data']['list']:
        temperatura = registro['main']['temp']
        if temperatura > umbral_temperatura:
            etiquetas.append(1)  # Hay ola de calor
        else:
            etiquetas.append(0)  # No hay ola de calor
    return etiquetas

# Función para cargar datos de instalaciones desde un archivo CSV y filtrar por tipo de instalación
def cargar_datos_instalaciones(nombre_archivo_csv, tipo_instalacion):
    csv.field_size_limit(500 * 1024)
    datos_instalaciones = []
    with open(nombre_archivo_csv, 'r', encoding='utf-8', newline='') as archivo_csv:
        lector_csv = csv.DictReader(archivo_csv)
        for fila in lector_csv:
            if fila['TIPO'] == tipo_instalacion:
                datos_instalaciones.append(fila)
    return datos_instalaciones

# Función para obtener recomendaciones de deportes utilizando el modelo de machine learning
def obtener_recomendaciones_deportes_modelo(datos_clima, etiquetas_ola_calor, datos_instalaciones):
    # Obtener los datos históricos de recomendaciones de deportes
    # Aquí asumimos que tienes un archivo CSV llamado 'datos_historicos_deportes.csv'
    # que contiene datos históricos con columnas como 'Temperatura', 'OlaDeCalor', 'TipoInstalacion' y 'Recomendacion'
    df_historico = pd.read_csv('datos_historicos_deportes.csv')

    # Características (features) del modelo: Temperatura y Ola de calor (1 si hay ola de calor, 0 si no)
    X = df_historico[['Temperatura', 'OlaDeCalor']].values

    # Etiquetas del modelo: Recomendación de deporte
    y = df_historico['Recomendacion'].values

    # Entrenar el modelo Random Forest
    modelo = RandomForestClassifier(random_state=42)
    modelo.fit(X, y)

    # Obtener los valores actuales de Temperatura y Ola de calor
    temperatura_actual = datos_clima['data']['list'][0]['main']['temp']
    ola_de_calor_actual = etiquetas_ola_calor[0]

    # Realizar la predicción del modelo con los datos actuales
    prediccion_actual = modelo.predict([[temperatura_actual, ola_de_calor_actual]])

    recomendaciones = []

    if prediccion_actual[0] == 1:
        # Si el modelo predice que hay que evitar deportes, entonces:
        # Recomendar actividades en centros de salud si tiene problemas de salud
        if tiene_problemas_salud:
            recomendaciones.extend(obtener_recomendaciones_centros_salud(datos_instalaciones))
    else:
        # Si el modelo predice que se pueden hacer deportes, entonces:
        # Recomendar actividades al aire libre en parques y jardines
        recomendaciones.extend(obtener_recomendaciones_parques(datos_instalaciones))

    return recomendaciones

# Función para obtener recomendaciones específicas para parques y jardines
def obtener_recomendaciones_parques(instalaciones):
    recomendaciones_parques = []
    for instalacion in instalaciones:
        if instalacion['TIPO'] == 'parque':
            recomendaciones_parques.append(f"Hacer ejercicio en el parque: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_parques

# Función para obtener recomendaciones específicas para centros de salud
def obtener_recomendaciones_centros_salud(instalaciones):
    recomendaciones_centros_salud = []
    for instalacion in instalaciones:
        if instalacion['TIPO'].lower() == 'centro_salud' and tiene_problemas_salud:
            recomendaciones_centros_salud.append(f"Realizar actividad en centro de salud: {instalacion['NOMBRE']} ({instalacion['DIRECCION']})")
    return recomendaciones_centros_salud

if __name__ == "__main__":
    nombre_archivo_json = "../Raw_Datasets/Trabajado/tiempo.json"  # Reemplaza con el nombre de tu archivo JSON
    nombre_archivo_sitios = "../Raw_Datasets/items_Giacomo/items.csv"  # Reemplaza con el nombre de tu archivo CSV de sitios (piscinas, parques y polideportivos)

    umbral_temperatura_ola_calor = 30.0  # Reemplaza con el umbral de temperatura para definir una ola de calor
    tiene_problemas_salud = True  # Reemplaza con True si el usuario tiene problemas de salud, False si no

    datos_clima = leer_datos_meteorologicos(nombre_archivo_json)
    etiquetas_ola_calor = etiquetar_ola_de_calor(datos_clima, umbral_temperatura_ola_calor)

    tipos_instalaciones_deportivas = {'PISCINA', 'ZONA VERDE', 'SENDA', 'CENTRO DE SALUD'}

    datos_instalaciones = []
    for tipo in tipos_instalaciones_deportivas:
        datos_instalaciones.extend(cargar_datos_instalaciones(nombre_archivo_sitios, tipo))

    # Obtener recomendaciones de deportes utilizando el modelo de machine learning
    recomendaciones_deportes_modelo = obtener_recomendaciones_deportes_modelo(
        datos_clima, etiquetas_ola_calor, datos_instalaciones
    )

    # Imprime las recomendaciones del modelo
    print("Recomendaciones de deportes (modelo):")
    for deporte in recomendaciones_deportes_modelo:
        print(deporte)


FileNotFoundError: [Errno 2] No such file or directory: 'datos_historicos_deportes.csv'

In [11]:
X = pd.read_csv('../Raw_Datasets/items_Giacomo/deportes_final.csv',sep=',')
X['Actividad'].unique()

array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia',
       'Entrenamiento en circuito', 'Levantamiento de peso',
       'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
       'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos',
       'Billar', 'Boxeo', 'Dardos', 'Frisbee', 'Ultimate frisbee', 'Golf',
       'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey', 'Petanca', 'Calva',
       'Artes marciales', 'Kickboxing', 'Kickball', 'Pádel', 'Raquetbol',
       'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje',
       'Fútbol', 'Fútbol sala', 'Sóftbol', 'Béisbol',
       'Tenis de mesa / ping pong', 'Tai chi', 'Tenis', 'Tenis en pareja',
       'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación',
       'Nado sincronizado', 'Waterpolo', 'Voleibol acuático',
       'Patinaje sobre hielo', 'Frontenis', 'Senderismo'], dtype=object)

In [36]:
import pandas as pd
import numpy as np
import random
import string
from datetime import datetime, timedelta

# Lista de deportes
deportes = np.array(['BMX', 'Ciclismo', 'Ciclismo estacionario', 'Calistenia', 'Entrenamiento en circuito',
                     'Levantamiento de peso', 'Máquina escaladora', 'Aeróbicos', 'Yoga', 'Aeróbicos acuáticos',
                     'Correr', 'Atletismo', 'Bádminton', 'Baloncesto', 'Bolos', 'Billar', 'Boxeo', 'Dardos',
                     'Frisbee', 'Ultimate frisbee', 'Golf', 'Minigolf', 'Gimnasia', 'Balonmano', 'Hockey',
                     'Petanca', 'Calva', 'Artes marciales', 'Kickboxing', 'Kickball', 'Pádel', 'Raquetbol',
                     'Escalada', 'Salto a la comba', 'Chito', 'Skateboard', 'Patinaje', 'Fútbol', 'Fútbol sala',
                     'Sóftbol', 'Béisbol', 'Tenis de mesa / ping pong', 'Tai chi', 'Tenis', 'Tenis en pareja',
                     'Voleibol', 'Marcha rápida', 'Montañismo', 'Caminar', 'Natación', 'Nado sincronizado',
                     'Waterpolo', 'Voleibol acuático', 'Patinaje sobre hielo', 'Frontenis', 'Senderismo'])

# Generar un DataFrame sintético con datos históricos de recomendaciones de deportes
np.random.seed(42)
num_datos = 1000

# Simular datos de Fecha y Hora
fechas = [datetime.now() - timedelta(days=i) for i in range(num_datos)]
horas = [random.randint(0, 23) for _ in range(num_datos)]
fechas_y_horas = [fecha.replace(hour=hora, minute=0, second=0) for fecha, hora in zip(fechas, horas)]

# Simular datos de Clima
clima = np.random.choice(['Nublado', 'Soleado', 'Lluvioso',], num_datos)

# Simular datos de Temperatura en el rango de 10 a 35 grados Celsius
temperatura = np.random.uniform(10, 35, num_datos)

# Simular datos de Humedad en el rango de 30 a 80 por ciento
humedad = np.random.randint(30, 81, num_datos)

# Simular datos de Velocidad del Viento en el rango de 0 a 20 km/h
velocidad_viento = np.random.randint(0, 21, num_datos)

# Simular datos de Edad en el rango de 18 a 65 años
edad = np.random.randint(18, 66, num_datos)

# Simular deportes que ha hecho antes
deportes_anteriores = np.random.choice(deportes, num_datos)

# Simular datos de distancia en el rango de 0.5 a 20 kilómetros
distancia = np.random.uniform(0.5, 20, num_datos)

# Generar nombres aleatorios para cada registro
def generar_nombre_aleatorio():
    return ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase) for _ in range(6))

nombres_aleatorios = [generar_nombre_aleatorio() for _ in range(num_datos)]

# Generar nombres de personas para cada registro
nombres_personas = np.random.choice(['John', 'Emily', 'Michael', 'Jessica', 'William', 'Sophia', 'Christopher', 'Emma', 'Daniel', 'Olivia'], num_datos)

# Crear el DataFrame
df_historico = pd.DataFrame({
    'Fecha': fechas_y_horas,
    'Nombres': nombres_personas,
    'Hora': horas,
    'Clima': clima,
    'Temperatura (°C)': temperatura.round().astype(int),
    'Humedad': humedad,
    'VelocidadViento': velocidad_viento,
    'Edad': edad,
    'Deporte_practicado': deportes_anteriores,
    'Distancia (Km)': distancia.round(2)
})

# Eliminar las columnas RecomendacionDeporte y RecomendacionTipoInstalacion
# df_historico.drop(columns=['RecomendacionDeporte', 'RecomendacionTipoInstalacion'], inplace=True)

# Guardar el DataFrame en un archivo CSV
df_historico.to_csv('../Raw_Datasets/datos_historicos_deportes.csv', index=False)


In [37]:
df_historico

,Fecha,Nombres,Hora,Clima,Temperatura (°C),Humedad,VelocidadViento,Edad,Deporte_practicado,Distancia (Km)
0,2023-07-28 05:00:00.479085,Sophia,5,Lluvioso,12,59,8,23,Waterpolo,2.98
1,2023-07-27 03:00:00.479085,Michael,3,Nublado,23,43,6,58,Bolos,4.19
2,2023-07-26 09:00:00.479085,William,9,Lluvioso,20,35,12,53,Voleibol,9.46
3,2023-07-25 17:00:00.479085,John,17,Lluvioso,35,32,10,19,Hockey,15.17
4,2023-07-24 22:00:00.479085,Christopher,22,Nublado,13,45,13,46,Golf,18.16
...,...,...,...,...,...,...,...,...,...,...
995,2020-11-05 23:00:00.480083,John,23,Soleado,33,46,14,38,Waterpolo,1.95
996,2020-11-04 10:00:00.480083,Emma,10,Soleado,13,53,9,50,Fútbol sala,2.75
997,2020-11-03 13:00:00.480083,Jessica,13,Lluvioso,30,50,8,55,Voleibol acuático,17.46
998,2020-11-02 22:00:00.480083,Olivia,22,Lluvioso,20,80,0,32,Tenis,9.71
